In [1]:
import sys
sys.path.append("..")
from Vocabulary import *

In [9]:
String = r"a b c d .       e, f g"

In [10]:
import re
def preprocess_line(text):
    text = re.sub(r' \\[^\s]{1,}','',text)
    text = re.sub(r' /[^\s]{1,}','',text)
    text = re.sub("\s?[^a-zA-Z\d\s:\u0004]","",text)
    return text

In [11]:
String = preprocess_line(String)
vocab = TaggedVocabulary(wordsWithoutTags = True)
vocab.build_from_text(String.split())
vocab.assignIds(shuffle=False)

In [12]:
class Co_Occurence_Capturer:

    def __init__(self):
        self.co_occurences = {}
        
    def _assign_entrys(self,word_ids,context_ids,dist,dynWindow):
        if not dynWindow:
            dist = 1.0
        for word_id in word_ids:
                for context_id in context_ids:
                    tuple = (word_id,context_id)
                    if tuple in self.co_occurences:
                        self.co_occurences[tuple] += 1.0 / dist
                    else:
                        self.co_occurences[tuple] = 1.0 / dist


    # Window lenght is one sided length
    # The window is applied on the left and the right.
    # A window size of 0 means, just the focus_word.
    # Article tokens is of type list
    def capture(self,vocab,article_tokens,window_size,dynWindow=True):
        print(article_tokens)
        article_ids = []
        for token in article_tokens:
            ids = vocab.get_ids_text(token)
            if ids == []:
                continue
            article_ids.append(ids)
    
        for focus_position,focus_ids in enumerate(article_ids):
            window_left = article_ids[max(0,focus_position-window_size):focus_position]
            for position,context_ids in enumerate(window_left):
                dist = abs(len(window_left) - position)
                self._assign_entrys(focus_ids,context_ids,dist,dynWindow) 
        
            window_right = article_ids[focus_position+1:focus_position+1+window_size]
            for position,context_ids in enumerate(window_right):
                dist = abs(1+ position)
                self._assign_entrys(focus_ids,context_ids,dist,dynWindow)

        return self.co_occurences
    
    def save_coocurrences(self,file_name):
        with open(file_name, 'wb') as file:
            cloudpickle.dump(self.co_occurences, file)
        self.co_occurences = {}

    def load_co_occurence(self,name):
        with open(name, 'rb+') as file:
            self.co_occurences = cloudpickle.load(file)

In [13]:
print(vocab.get_size())

7


In [14]:
print(vocab.word2Id)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6}


In [15]:
#String = r"a b c d . e, f g"
#print(re.sub("\s?[^a-zA-Z\d\s:\u0004]","",String))
print(String)

a b c d       e f g


In [16]:
co = Co_Occurence_Capturer()
tmp = co.capture(vocab,String.split(),3,dynWindow=False)
print(tmp)


['a', 'b', 'c', 'd', 'e', 'f', 'g']
{(0, 1): 1.0, (0, 2): 1.0, (0, 3): 1.0, (1, 0): 1.0, (1, 2): 1.0, (1, 3): 1.0, (1, 4): 1.0, (2, 0): 1.0, (2, 1): 1.0, (2, 3): 1.0, (2, 4): 1.0, (2, 5): 1.0, (3, 0): 1.0, (3, 1): 1.0, (3, 2): 1.0, (3, 4): 1.0, (3, 5): 1.0, (3, 6): 1.0, (4, 1): 1.0, (4, 2): 1.0, (4, 3): 1.0, (4, 5): 1.0, (4, 6): 1.0, (5, 2): 1.0, (5, 3): 1.0, (5, 4): 1.0, (5, 6): 1.0, (6, 3): 1.0, (6, 4): 1.0, (6, 5): 1.0}
